In [31]:
!pip install kfp==1.8.9
!pip install google-cloud-pipeline-components==0.2.0

In [10]:
import kfp
from kfp import components
import kfp.dsl as dsl
from kfp.v2 import compiler
from kfp.v2.google.client import AIPlatformClient
from google_cloud_pipeline_components import aiplatform as gcc_aip

from components.preprocessing import split_data
from components.training import xgb_training
from pipelines.pipeline import xgb_pipeline

In [11]:
compiler.Compiler().compile(
    pipeline_func=xgb_pipeline,
    package_path="pl.json"
)


In [13]:
from google.cloud.aiplatform.pipeline_jobs import PipelineJob

pl = PipelineJob(display_name= 'xgb-job',
        template_path= "pl.json",
        location='us-central1',
        pipeline_root="gs://feature-store-mars21/vertex-xgb-ml-ops/a3b8252",
        parameter_values={'project_id': 'feature-store-mars21', 
                          'data_path': 'gs://mortgage_dataset_files/mortgage-small.csv'})

pl.run(sync=False)

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
